In [56]:
import wget

In [8]:
!python -m wget https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt

/anaconda2/bin/python: No module named wget


In [5]:
!wget

/bin/bash: wget: command not found


In [19]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

In [22]:
wget.download(url)

100% [......................................................] 1115394 / 1115394

'input.txt'

In [23]:
!ls

ML_for_Calo_students.ipynb selfAttention.ipynb
README.md


In [24]:
with open("../input.txt", 'r', encoding ='utf-8') as f:
    text = f.read()

In [25]:
len(text)

1115394

In [26]:
#print 1st 1000 characters
print("print 1st 1000 characters: ",text[:1000])

print 1st 1000 characters:  First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, 

In [29]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [30]:
#create mapping from characters to integer
ctoi = {ch:i for i,ch in enumerate(chars)}
itoc = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[ctoi[c] for c in s]
decode = lambda s:"".join(itoc[i] for i in s)

In [31]:
encode("hi there")

[46, 47, 1, 58, 46, 43, 56, 43]

In [32]:
decode(encode("hi there"))

'hi there'

In [43]:
import torch
torch.manual_seed(1337)

In [35]:
data = torch.tensor(encode(text), dtype = torch.long)

In [36]:
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [37]:
data[:1000]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [38]:
n = int(0.9*len(data))
train_data = data[:n]
valid_data = data[n:]

In [93]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [69]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when context is {context} target is {target}")

when context is tensor([18]) target is 47
when context is tensor([18, 47]) target is 56
when context is tensor([18, 47, 56]) target is 57
when context is tensor([18, 47, 56, 57]) target is 58
when context is tensor([18, 47, 56, 57, 58]) target is 1
when context is tensor([18, 47, 56, 57, 58,  1]) target is 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [70]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print("ix: ",ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    #print("x.shape: ",x.shape)
    #print("y.shape: ",y.shape)
    return x, y

In [71]:
get_batch("train")

(tensor([[47, 43, 52, 42,  6,  1, 61, 43],
         [47, 52, 45,  1, 58, 46, 43,  1],
         [50, 47, 43, 56,  1, 56, 43, 48],
         [53, 58, 46,  1, 47, 52,  1, 63]]),
 tensor([[43, 52, 42,  6,  1, 61, 43,  1],
         [52, 45,  1, 58, 46, 43,  1, 56],
         [47, 43, 56,  1, 56, 43, 48, 53],
         [58, 46,  1, 47, 52,  1, 63, 53]]))

In [72]:
import torch.nn as nn
from torch.nn import functional as F

In [77]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        #print("logits: ",logits)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [78]:
xb, yb = get_batch('train')

In [79]:
model = BigramLanguageModel(vocab_size)

In [89]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [95]:
for iter in range(100):

    # every once in a while evaluate the loss on train and val sets
    #if iter % eval_interval == 0:
    #    losses = estimate_loss()
    #    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    #print(loss.item())

In [80]:
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.6710, grad_fn=<NllLossBackward0>)


In [87]:
# generate from the model
context = torch.zeros((2, 1), dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=500)[1].tolist()))


x, D'hJG-kkmMZ.MR?k.3igemYKsGMXlArwxJtaUbA&Mn:C-aJU bUyqA.
xjh:JkMnlzCjhcyb:PvfP-IED
m szEHKiRfSV.SfgznKOCZL',?aA-o:KdXLq,-B$LSLRYm;??tcSFOBoYS.LCjl'nMpigDLBVnt.LR:KMSCXJMJZZLVCl;JfqR?XtPQ!'TTYt:b!it-V'pS!
:gS3ghOKjoVOaDJ&3a3XKjuPSK-jJaEE;?TprpS-pdijfjXlQCI
mYgZKCIXfNrJaOm&gNdXj&q3mzz:Z3;hJszag:sgKp$mQC,Xw$eYUjiOiwG3Lt$QqUb!?GNNIUbk.yvE N
RcfPlQQN&s.MfgjZXKFAHIodcfPPMQsV
r -hyb!TMJ-
$?mLnZoJTf sXwsp?sYNPP!gExeE fPO'dXd:QBTKnl;uCETPBQCM&!nlk:RX.&Bjm:MSvfptpEb!lkWX.ynsaGrDgKuuUQtNe,VYJaEkQQ:TpaEiV
